In [ ]:
%%capture
# Allowed to make changes.

# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

# All important installations and directory configurations
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install sentence-transformers
!mkdir models
%cd models
init_file="""
# from .modeling_electra import (
#     ElectraConfig,
#     ElectraForSequenceClassification,
#     ElectraForQuestionAnsweringAVPool
# )
import torch
from torch import nn
from torch.nn import CrossEntropyLoss

from transformers import (
    ElectraForSequenceClassification as SeqClassification,
    ElectraPreTrainedModel,
    ElectraModel,
    ElectraConfig,
)

# from .modeling_outputs import (
#     QuestionAnsweringModelOutput,
#     QuestionAnsweringNaModelOutput,
# )

from typing import Optional, Tuple

import torch

from dataclasses import dataclass
from transformers.file_utils import ModelOutput
from transformers.modeling_outputs import QuestionAnsweringModelOutput


@dataclass
class QuestionAnsweringNaModelOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    start_logits: torch.FloatTensor = None
    end_logits: torch.FloatTensor = None
    has_logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


class ElectraForSequenceClassification(SeqClassification):
    model_type = "electra"


class ElectraForQuestionAnsweringAVPool(ElectraPreTrainedModel):
    config_class = ElectraConfig
    base_model_prefix = "electra"
    model_type = "electra"

    def __init__(self, config):
        super(ElectraForQuestionAnsweringAVPool, self).__init__(config)
        self.num_labels = config.num_labels

        self.electra  = ElectraModel(config)
        self.qa_outputs = nn.Linear(config.hidden_size, config.num_labels)
        self.has_ans = nn.Sequential(
            nn.Dropout(p=config.hidden_dropout_prob),
            nn.Linear(config.hidden_size, 2)
        )

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        start_positions=None,
        end_positions=None,
        is_impossibles=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        discriminator_hidden_states = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )

        sequence_output = discriminator_hidden_states[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        first_word = sequence_output[:, 0, :]

        has_logits = self.has_ans(first_word)

        total_loss = None
        if (
            start_positions is not None and
            end_positions is not None and
            is_impossibles is not None
        ):
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            if len(is_impossibles.size()) > 1:
                is_impossibles = is_impossibles.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions.clamp_(0, ignored_index)
            end_positions.clamp_(0, ignored_index)
            is_impossibles.clamp_(0, ignored_index)

            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            span_loss = start_loss + end_loss

            # Internal Front Verification (I-FV)
            # alpha1 == 1.0, alpha2 == 0.5
            choice_loss = loss_fct(has_logits, is_impossibles.long())
            total_loss = 1.0 * span_loss + 0.5 * choice_loss

        if not return_dict:
            output = (
                start_logits,
                end_logits,
                has_logits,
            ) + discriminator_hidden_states[2:] # hidden_states, attentions
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringNaModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            has_logits=has_logits,
            hidden_states=discriminator_hidden_states.hidden_states,
            attentions=discriminator_hidden_states.attentions,
        )
"""
electra_model="""
import torch
from torch import nn
from torch.nn import CrossEntropyLoss

from transformers import (
    ElectraForSequenceClassification as SeqClassification,
    ElectraPreTrainedModel,
    ElectraModel,
    ElectraConfig,
)

from .modeling_outputs import (
    QuestionAnsweringModelOutput,
    QuestionAnsweringNaModelOutput,
)


class ElectraForSequenceClassification(SeqClassification):
    model_type = "electra"


class ElectraForQuestionAnsweringAVPool(ElectraPreTrainedModel):
    config_class = ElectraConfig
    base_model_prefix = "electra"
    model_type = "electra"

    def __init__(self, config):
        super(ElectraForQuestionAnsweringAVPool, self).__init__(config)
        self.num_labels = config.num_labels

        self.electra  = ElectraModel(config)
        self.qa_outputs = nn.Linear(config.hidden_size, config.num_labels)
        self.has_ans = nn.Sequential(
            nn.Dropout(p=config.hidden_dropout_prob),
            nn.Linear(config.hidden_size, 2)
        )

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        start_positions=None,
        end_positions=None,
        is_impossibles=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        discriminator_hidden_states = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )

        sequence_output = discriminator_hidden_states[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        first_word = sequence_output[:, 0, :]

        has_logits = self.has_ans(first_word)

        total_loss = None
        if (
            start_positions is not None and
            end_positions is not None and
            is_impossibles is not None
        ):
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            if len(is_impossibles.size()) > 1:
                is_impossibles = is_impossibles.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions.clamp_(0, ignored_index)
            end_positions.clamp_(0, ignored_index)
            is_impossibles.clamp_(0, ignored_index)

            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            span_loss = start_loss + end_loss

            # Internal Front Verification (I-FV)
            # alpha1 == 1.0, alpha2 == 0.5
            choice_loss = loss_fct(has_logits, is_impossibles.long())
            total_loss = 1.0 * span_loss + 0.5 * choice_loss

        if not return_dict:
            output = (
                start_logits,
                end_logits,
                has_logits,
            ) + discriminator_hidden_states[2:] # hidden_states, attentions
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringNaModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            has_logits=has_logits,
            hidden_states=discriminator_hidden_states.hidden_states,
            attentions=discriminator_hidden_states.attentions,
        )
"""
electra_outputs="""
from typing import Optional, Tuple

import torch

from dataclasses import dataclass
from transformers.file_utils import ModelOutput
from transformers.modeling_outputs import QuestionAnsweringModelOutput


@dataclass
class QuestionAnsweringNaModelOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    start_logits: torch.FloatTensor = None
    end_logits: torch.FloatTensor = None
    has_logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

"""

file = open("__init__.py", "w")
n = file.write(init_file)
file.close()

file = open("modeling_electra.py", "w")
n = file.write(electra_model)
file.close()

file = open("modeling_outputs.py", "w")
n = file.write(electra_outputs)
file.close()

%cd ..


# Importing all libraries
import collections
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval
import numpy as np
import os
import gc
import time
import math
import yaml
import torch
import evaluate
import sys
import dataclasses
import re
import copy
import argparse
import datasets
import spacy
import pickle

%cd /content/
default_config="""

"""
data=yaml.safe_load(default_config)

with open("default_config.yaml", "w") as fh:
  yaml.dump(data, fh)

from tqdm import tqdm
from datetime import datetime
from typing import Optional, List, Dict, Tuple, Callable, Any, Union, Iterable, NewType
from datasets import Sequence, Value, Features
from datasets import Dataset, DatasetDict
from transformers.tokenization_utils_fast import PreTrainedTokenizerFast
from gensim.summarization.bm25 import BM25
import transformers as tr
import sklearn as sl
import sentence_transformers as st
from contextlib import suppress
import warnings
from nltk.corpus import stopwords
warnings.filterwarnings("ignore")

from transformers import (
    is_datasets_available,
    is_torch_tpu_available,
    AutoTokenizer,
)

from transformers.trainer_utils import (
    PredictionOutput,
    EvalPrediction,
    EvalLoopOutput,
    denumpify_detensorize,
    speed_metrics,
    # logging
)

from transformers.utils import logging
from transformers.debug_utils import DebugOption

from transformers.hf_argparser import HfArgumentParser as ArgumentParser
from pathlib import Path
from dataclasses import dataclass, field
from transformers import TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification

if is_datasets_available():
    import datasets

if is_torch_tpu_available():
    import torch_xla.core.xla_model as xm
    import torch_xla.debug.metrics as met

from transformers import Trainer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from dataclasses import dataclass, field
import models


logger = logging.get_logger(__name__)

%cd /content/drive/MyDrive/FinalCompilation


In [ ]:
sentence_model=st.SentenceTransformer('distilbert-base-nli-mean-tokens')
english_pipeline=spacy.load("en_core_web_sm")
tokenizer=tr.BertTokenizer.from_pretrained("bert-base-uncased")
normal_model=tr.BertModel.from_pretrained("bert-base-uncased")
questionGenModel = tr.T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
questionGenTokenizer = tr.AutoTokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
tfidf = TfidfVectorizer()

In [ ]:
def tokens(sentence):

  tokenVal = tokenizer.tokenize(word)
  tokenID = tokenizer.convert_tokens_to_ids(tokenVal)
  segID = [1]* len(tokenVal)

  torchVal = torch.tensor([tokenID])
  torchSeg = torch.tensor([segID])

  return normal_model(torchVal, torchSeg)[-1].detach().numpy()

def vectorization(sentence):
  vectors = sl.feature_extraction.text.CountVectorizer(ngram_range=(1,1), stop_words="english").fit([sentence])
  return vectors.get_feature_names()

def positions(paragraph):
  sentence=english_pipeline(paragraph)
  sentences=[word.pos_ for word in sentence]
  return sentences,paragraph.split()

def Sentences(paragraph):
  sentence = english_pipeline(paragraph)
  return list(sentence.sents)

import random
def answerGen(paragraph, module_type = 't'):
  answers = []
  num_ans = 1
  print('2')
  for word in Sentences(paragraph):
    print('3')
    # print(str(word))
    if(len(str(word))<=15):
      print("4")
      continue
    print('5')
    ans = vectorization(str(word))
    # print(f'vectors : {ans}')
    if module_type == 't':
      sentenceToken = tokens(str(word))
      ansToken = tokens(' '.join(ans))
    else:
      print("6")
      sentenceToken = sentence_model.encode([str(word)])
      ansToken = sentence_model.encode(ans)
    print('7')
    vals = sl.metrics.pairwise.cosine_similarity(sentenceToken, ansToken)
    # print(vals)
    answers += [(ans[value], str(word)) for value in vals.argsort()[0][-num_ans:]]
    print(answers)
  return random.sample(answers,int(len(answers)))

def questionGen(context, ans):

  questionGenEncode = questionGenTokenizer.encode_plus("context: {} answer: {}".format(context,ans),max_length=256, pad_to_max_length=False,truncation=True, return_tensors="pt")

  questionGenOutput = questionGenModel.generate(input_ids=questionGenEncode["input_ids"],
                                  attention_mask=questionGenEncode["attention_mask"],
                                  early_stopping=True,
                                  num_beams=1,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=1,
                                  max_length=100);


  final = [questionGenTokenizer.decode(i,skip_special_tokens=True) for i in questionGenOutput];


  questionGenerated = (final[0].replace("question:","")).strip();
  print(type(questionGenerated))
  return questionGenerated

from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
model_name = "deepset/tinyroberta-squad2"
nlpgen = pipeline('question-answering', model=model_name, tokenizer=model_name)



def syn_data_former(df):
  all_questions_for_paragraph={}
  for para in range(len(df['paragraph'])):
    paragraph=df['paragraph'][para]
    print(paragraph, df['id'][para])
    # with suppress(Exception):
    all_questions_for_paragraph[df['id'][para]]=[]
    for ans,context in answerGen(paragraph,"st"):
      print('flag')
      question=questionGen(context,ans)
      QA_input = {
      'question': question,
      'context': paragraph,
                  }
      res = nlpgen(QA_input)
      all_questions_for_paragraph[df['id'][para]].append({question:res})
  return all_questions_for_paragraph

def comparisons(question,id,dictionary):

  questions=list(dictionary[id][0].keys())
  current_question=question
  vectors = tfidf.fit_transform(questions + [current_question])

  question_vector=vectors[-1]
  similarities = cosine_similarity(question_vector, vectors[:-1])

  index=similarities.argmax()
  score=similarities[index]

  if(score>=0.6):
    return score, dictionary[id][0][questions[index]]['answer']
  else:
    return None, None

In [ ]:
%%capture
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.
final_pred = []
data_para = pd.read_csv("/content/drive/MyDrive/paragraphs.csv")

class ToMixin:

    def _optimizer_to(self, device: str = "cpu"):
        # https://github.com/pytorch/pytorch/issues/8741
        for param in self.optimizer.state.values():
            # Not sure there are any global tensors in the state dict
            if isinstance(param, torch.Tensor):
                param.data = param.data.to(device)
                if param._grad is not None:
                    param._grad.data = param._grad.data.to(device)
            elif isinstance(param, dict):
                for subparam in param.values():
                    if isinstance(subparam, torch.Tensor):
                        subparam.data = subparam.data.to(device)
                        if subparam._grad is not None:
                            subparam._grad.data = subparam._grad.data.to(
                                device)

    def _scheduler_to(self, device: str = "cpu"):
        # https://github.com/pytorch/pytorch/issues/8741
        for param in self.lr_scheduler.__dict__.values():
            if isinstance(param, torch.Tensor):
                param.data = param.data.to(device)
                if param._grad is not None:
                    param._grad.data = param._grad.data.to(device)


class BaseReader(Trainer, ToMixin):
    name: str = None

    def __init__(
        self,
        *args,
        data_args = {},
        eval_examples: datasets.Dataset = None,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.data_args = data_args
        self.eval_examples = eval_examples

    def free_memory(self):
        self.model.to("cpu")
        self._optimizer_to("cpu")
        self._scheduler_to("cpu")
        torch.cuda.empty_cache()
        gc.collect()

    def postprocess(
        self,
        output: EvalLoopOutput,
    ) -> Union[Any, EvalPrediction]:
        return output

    def evaluate(
        self,
        eval_dataset: Optional[datasets.Dataset] = None,
        eval_examples: Optional[datasets.Dataset] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ) -> Dict[str, float]:
        # memory metrics - must set up as early as possible
        self._memory_tracker.start()
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        start_time = time.time()
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        try:
            output = eval_loop(
                eval_dataloader,
                description="Evaluation",
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics

        if isinstance(eval_dataset, datasets.Dataset):
            eval_dataset.set_format(
                type=eval_dataset.format["type"],
                columns=list(eval_dataset.features.keys()),
            )

        eval_preds = self.postprocess(output, eval_examples, eval_dataset, mode="evaluate")

        metrics = {}
        if self.compute_metrics is not None:
            metrics = self.compute_metrics(eval_preds)
            # To be JSON-serializable, we need to remove numpy types or zero-d tensors
            metrics = denumpify_detensorize(metrics)

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)

            total_batch_size = self.args.eval_batch_size * self.args.world_size
            metrics.update(
                speed_metrics(
                    metric_key_prefix,
                    start_time,
                    num_samples=output.num_samples,
                    num_steps=math.ceil(output.num_samples / total_batch_size),
                )
            )
            self.log(metrics)

        # Log and save evaluation results
        filename = "eval_results.txt"
        eval_result_file = self.name + "_" + filename if self.name else filename
        with open(os.path.join(self.args.output_dir, eval_result_file), "a")  as writer:
            logger.info("***** Eval results *****")
            writer.write("***** Eval results *****\n")
            writer.write(f"{datetime.now()}")
            for key in sorted(metrics.keys()):
                logger.info("  %s = %s", key, str(metrics[key]))
                writer.write("%s = %s\n" % (key, str(metrics[key])))
            writer.write("\n")

        if DebugOption.TPU_METRICS_DEBUG in self.args.debug:
            # tpu-comment: PyTorch/XLA에 대한 Logging debug metrics (compile, execute times, ops, etc.)
            xm.master_print(met.metrics_report())

        self.control = self.callback_handler.on_evaluate(
            self.args, self.state, self.control, metrics
        )

        self._memory_tracker.stop_and_update_metrics(metrics)

        return metrics

    def predict(
        self,
        test_dataset: datasets.Dataset,
        test_examples: datasets.Dataset,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "test",
        mode: bool = "predict",
    ) -> PredictionOutput:
        # memory metrics - must set up as early as possible
        self._memory_tracker.start()

        test_dataloader = self.get_test_dataloader(test_dataset)
        start_time = time.time()

        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        try:
            output = eval_loop(
                test_dataloader,
                description="Prediction",
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics

        if isinstance(test_dataset, datasets.Dataset):
            test_dataset.set_format(
                type=test_dataset.format["type"],
                columns=list(test_dataset.features.keys()),
            )

        predictions = self.postprocess(output, test_examples, test_dataset, mode=mode)

        self._memory_tracker.stop_and_update_metrics(output.metrics)

        return predictions


EXAMPLE_FEATURES = Features(
    {
        "guid": Value(dtype="string", id=None),
        "question": Value(dtype="string", id=None),
        "context": Value(dtype="string", id=None),
        "answers": Sequence(
            feature={
                "text": Value(dtype="string", id=None),
                "answer_start": Value(dtype="int32", id=None),
            },
        ),
        "is_impossible": Value(dtype="bool", id=None),
        "title": Value(dtype="string", id=None),
        "classtype": Value(dtype="string", id=None),
        "source": Value(dtype="string", id=None),
        "dataset": Value(dtype="string", id=None),
    }
)

SKETCH_TRAIN_FEATURES = Features(
    {
        "input_ids": Sequence(feature=Value(dtype='int32', id=None)),
        "attention_mask": Sequence(feature=Value(dtype='int8', id=None)),
        "token_type_ids": Sequence(feature=Value(dtype='int8', id=None)),
        "labels": Value(dtype='int64', id=None),
    }
)

SKETCH_EVAL_FEATURES = Features(
    {
        "input_ids": Sequence(feature=Value(dtype='int32', id=None)),
        "attention_mask": Sequence(feature=Value(dtype='int8', id=None)),
        "token_type_ids": Sequence(feature=Value(dtype='int8', id=None)),
        "labels": Value(dtype='int64', id=None),
        "example_id": Value(dtype='string', id=None),
    }
)

INTENSIVE_TRAIN_FEATUERS = Features(
    {
        "input_ids": Sequence(feature=Value(dtype='int32', id=None)),
        "attention_mask": Sequence(feature=Value(dtype='int8', id=None)),
        "token_type_ids": Sequence(feature=Value(dtype='int8', id=None)),
        "start_positions": Value(dtype='int64', id=None),
        "end_positions": Value(dtype='int64', id=None),
        "is_impossibles": Value(dtype='float64', id=None),
    }
)

INTENSIVE_EVAL_FEATUERS = Features(
    {
        "input_ids": Sequence(feature=Value(dtype='int32', id=None)),
        "attention_mask": Sequence(feature=Value(dtype='int8', id=None)),
        "token_type_ids": Sequence(feature=Value(dtype='int8', id=None)),
        "offset_mapping": Sequence(
            feature=Sequence(
                feature=Value(dtype='int64', id=None)
            )
        ),
        "example_id": Value(dtype='string', id=None),
    }
)

QUESTION_COLUMN_NAME = "question"
CONTEXT_COLUMN_NAME = "context"
ANSWER_COLUMN_NAME = "answers"
ANSWERABLE_COLUMN_NAME = "is_impossible"
ID_COLUMN_NAME = "guid"

SCORE_EXT_FILE_NAME = "cls_score.json"
INTENSIVE_PRED_FILE_NAME = "predictions.json"
NBEST_PRED_FILE_NAME = "nbest_predictions.json"
SCORE_DIFF_FILE_NAME = "null_odds.json"

accuracy = evaluate.load("accuracy").compute
precision = evaluate.load("precision").compute
recall = evaluate.load("recall").compute
f1 = evaluate.load("f1").compute
squad_v2 = evaluate.load("squad_v2").compute


def compute_classification_metric(p: EvalPrediction):
    """
    'predictions': Value(dtype='int32', id=None)
    'references': Value(dtype='int32', id=None)
    """
    predictions = p.predictions.argmax(axis=1)
    references = p.label_ids
    metric = accuracy(predictions=predictions, references=references)
    metric.update(precision(predictions=predictions, references=references))
    metric.update(recall(predictions=predictions, references=references))
    metric.update(f1(predictions=predictions, references=references))
    return metric


def compute_squad_v2(p: EvalPrediction):
    """
    'predictions': {
        'id': Value(dtype='string', id=None),
        'prediction_text': Value(dtype='string', id=None),
        'no_answer_probability': Value(dtype='float32', id=None)
    }
    'references': {
        'id': Value(dtype='string', id=None),
        'answers': Sequence(
            feature={
                'text': Value(dtype='string', id=None),
                'answer_start': Value(dtype='int32', id=None)
            },
            length=-1, id=None
        )
    }
    """
    predictions = p.predictions
    references = p.label_ids
    return squad_v2(predictions=predictions, references=references)


def get_sketch_features(tokenizer, mode, data_args):

    pad_on_right = tokenizer.padding_side == "right"
    max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

    def tokenize_fn(examples):

        tokenized_examples = tokenizer(
            examples[QUESTION_COLUMN_NAME if pad_on_right else CONTEXT_COLUMN_NAME],
            examples[CONTEXT_COLUMN_NAME if pad_on_right else QUESTION_COLUMN_NAME],

            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=data_args.doc_stride,

            return_overflowing_tokens=True,
            return_offsets_mapping=False,

            return_token_type_ids=data_args.return_token_type_ids,
            padding="max_length" if data_args.pad_to_max_length else False,

        )
        return tokenized_examples

    def prepare_train_features(examples):
        tokenized_examples = tokenize_fn(examples)
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        tokenized_examples["labels"] = []

        for i in range(len(tokenized_examples["input_ids"])):
            sample_index = sample_mapping[i]


            is_impossible = examples[ANSWERABLE_COLUMN_NAME][sample_index]
            tokenized_examples["labels"].append(0 if not is_impossible else 1)

        return tokenized_examples

    def prepare_eval_features(examples):
        tokenized_examples = tokenize_fn(examples)
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        tokenized_examples["example_id"] = []
        tokenized_examples["labels"] = []

        for i in range(len(tokenized_examples["input_ids"])):

            sample_index = sample_mapping[i]

            id_col = examples[ID_COLUMN_NAME][sample_index]
            tokenized_examples["example_id"].append(id_col)


            is_impossible = examples[ANSWERABLE_COLUMN_NAME][sample_index]
            tokenized_examples["labels"].append(0 if not is_impossible else 1)

        return tokenized_examples

    def prepare_test_features(examples):
        tokenized_examples = tokenize_fn(examples)
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        tokenized_examples["example_id"] = []

        for i in range(len(tokenized_examples["input_ids"])):

            sample_index = sample_mapping[i]

            id_col = examples[ID_COLUMN_NAME][sample_index]
            tokenized_examples["example_id"].append(id_col)

        return tokenized_examples

    if mode == "train":
        get_features_fn = prepare_train_features
    elif mode == "eval":
        get_features_fn = prepare_eval_features
    elif mode == "test":
        get_features_fn = prepare_test_features

    return get_features_fn, True


def get_intensive_features(tokenizer, mode, data_args):

    pad_on_right = tokenizer.padding_side == "right"
    max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)
    beam_based = data_args.intensive_model_type in ["xlnet", "xlm"]

    def tokenize_fn(examples):

        tokenized_examples = tokenizer(
            examples[QUESTION_COLUMN_NAME if pad_on_right else CONTEXT_COLUMN_NAME],
            examples[CONTEXT_COLUMN_NAME if pad_on_right else QUESTION_COLUMN_NAME],

            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=data_args.doc_stride,

            return_overflowing_tokens=True,

            return_offsets_mapping=True,

            return_token_type_ids=data_args.return_token_type_ids,
            padding="max_length" if data_args.pad_to_max_length else False,

        )
        return tokenized_examples

    def prepare_train_features(examples):
        tokenized_examples = tokenize_fn(examples)

        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        offset_mapping = tokenized_examples.pop("offset_mapping")

        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []
        tokenized_examples["is_impossibles"] = []
        if beam_based:
            tokenized_examples["cls_index"] = []
            tokenized_examples["p_mask"] = []

        for i, offsets in enumerate(offset_mapping):

            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)

            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0

            if beam_based:
                tokenized_examples["cls_index"].append(cls_index)
                tokenized_examples["p_mask"].append(
                    [
                        0.0 if s == context_index or k == cls_index else 1.0
                        for k, s in enumerate(sequence_ids)
                    ]
                )

            sample_index = sample_mapping[i]
            answers = examples[ANSWER_COLUMN_NAME][sample_index]
            is_impossible = examples[ANSWERABLE_COLUMN_NAME][sample_index]

            if is_impossible or len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
                tokenized_examples["is_impossibles"].append(1.0)
            else:
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                token_start_index = 0
                while sequence_ids[token_start_index] != context_index:
                    token_start_index += 1

                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != context_index:
                    token_end_index -= 1

                if not (
                    offsets[token_start_index][0] <= start_char and
                    offsets[token_end_index][1] >= end_char
                ):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                    tokenized_examples["is_impossibles"].append(1.0)
                else:

                    while (
                        token_start_index < len(offsets) and
                        offsets[token_start_index][0] <= start_char
                    ):
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)

                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

                    tokenized_examples["is_impossibles"].append(0.0) # answerable

        return tokenized_examples

    def prepare_eval_features(examples):
        tokenized_examples = tokenize_fn(examples)

        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        tokenized_examples["example_id"] = []


        if beam_based:
            tokenized_examples["cls_index"] = []
            tokenized_examples["p_mask"] = []

        for i, input_ids in enumerate(tokenized_examples["input_ids"]):

            cls_index = input_ids.index(tokenizer.cls_token_id)

            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0

            if beam_based:
                tokenized_examples["cls_index"].append(cls_index)
                tokenized_examples["p_mask"].append(
                    [
                        0.0 if s == context_index or k == cls_index else 1.0
                        for k, s in enumerate(sequence_ids)
                    ]
                )

            sample_index = sample_mapping[i]
            id_col = examples[ID_COLUMN_NAME][sample_index]
            tokenized_examples["example_id"].append(id_col)

            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]

        return tokenized_examples

    if mode == "train":
        get_features_fn = prepare_train_features
    elif mode == "eval":
        get_features_fn = prepare_eval_features
    elif mode == "test":
        get_features_fn = prepare_eval_features

    return get_features_fn, True

sys.path.append('/content/models')

@dataclass
class RetroDataModelArguments:
    pass


@dataclass
class DataArguments(RetroDataModelArguments):
    max_seq_length: int = field(
        default=512,
        metadata={"help": ""},
    )
    max_answer_length: int = field(
        default=30,
        metadata={"help": ""},
    )
    doc_stride: int = field(
        default=128,
        metadata={"help": ""},
    )
    return_token_type_ids: bool = field(
        default=True,
        metadata={"help": ""},
    )
    pad_to_max_length: bool = field(
        default=True,
        metadata={"help": ""},
    )
    preprocessing_num_workers: int = field(
        default=5,
        metadata={"help": ""},
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={"help": ""},
    )
    version_2_with_negative: bool = field(
        default=True,
        metadata={"help": ""},
    )
    null_score_diff_threshold: float = field(
        default=0.0,
        metadata={"help": ""},
    )
    rear_threshold: float = field(
        default=0.0,
        metadata={"help": ""},
    )
    n_best_size: int = field(
        default=20,
        metadata={"help": ""},
    )
    use_choice_logits: bool = field(
        default=False,
        metadata={"help": ""},
    )
    start_n_top: int = field(
        default=-1,
        metadata={"help": ""},
    )
    end_n_top: int = field(
        default=-1,
        metadata={"help": ""},
    )
    beta1: int = field(
        default=1,
        metadata={"help": ""},
    )
    beta2: int = field(
        default=1,
        metadata={"help": ""},
    )
    best_cof: int = field(
        default=1,
        metadata={"help": ""},
    )


@dataclass
class ModelArguments(RetroDataModelArguments):
    use_auth_token: bool = field(
        default=False,
        metadata={"help": ""},
    )


@dataclass
class SketchModelArguments(ModelArguments):
    sketch_revision: str = field(
        default="main",
        metadata={"help": ""},
    )
    sketch_model_name: str = field(
        default="monologg/koelectra-small-v3-discriminator",
        metadata={"help": ""},
    )
    sketch_tokenizer_name: str = field(
        default=None,
        metadata={"help": ""},
    )
    sketch_architectures: str = field(
        default="ElectraForSequenceClassification",
        metadata={"help": ""},
    )


@dataclass
class IntensiveModelArguments(ModelArguments):
    intensive_revision: str = field(
        default="main",
        metadata={"help": ""},
    )
    intensive_model_name: str = field(
        default="monologg/koelectra-small-v3-discriminator",
        metadata={"help": ""},
    )
    intensive_tokenizer_name: str = field(
        default=None,
        metadata={"help": ""},
    )
    intensive_architectures: str = field(
        default="ElectraForQuestionAnsweringAVPool",
        metadata={"help": ""},
    )


@dataclass
class RetroArguments(
    DataArguments,
    SketchModelArguments,
    IntensiveModelArguments,
):
    def __post_init__(self):
        # Sketch
        self.intensive_model_name = self.intensive_model_name + "/" + self.intensive_revision
        self.sketch_model_name =  self.sketch_model_name  + "/" + self.sketch_revision
        model_cls = getattr(models, self.sketch_architectures, None)
        if model_cls is None:
            raise AttributeError
        self.sketch_model_cls = model_cls
        self.sketch_model_type = model_cls.model_type
        if self.sketch_tokenizer_name is None:
            self.sketch_tokenizer_name = self.sketch_model_name
        # Intensive
        model_cls = getattr(models, self.intensive_architectures, None)
        if model_cls is None:
            raise AttributeError
        self.intensive_model_cls = model_cls
        self.intensive_model_type = model_cls.model_type
        if self.intensive_tokenizer_name is None:
            self.intensive_tokenizer_name = self.intensive_model_name



DataClass = NewType("DataClass", Any)
DataClassType = NewType("DataClassType", Any)


def lambda_field(default, **kwargs):
    return field(default_factory=lambda: copy.copy(default))


class HfArgumentParser(ArgumentParser):

    def parse_yaml_file(self, yaml_file: str) -> Tuple[DataClass, ...]:
        """
        Alternative helper method that does not use `argparse` at all, instead loading a yaml file and populating the
        dataclass types.
        """
        # https://stackoverflow.com/questions/30458977/yaml-loads-5e-6-as-string-and-not-a-number
        loader = yaml.SafeLoader
        loader.add_implicit_resolver(
            u'tag:yaml.org,2002:float',
            re.compile(u'''^(?:
             [-+]?(?:[0-9][0-9_]*)\\.[0-9_]*(?:[eE][-+]?[0-9]+)?
            |[-+]?(?:[0-9][0-9_]*)(?:[eE][-+]?[0-9]+)
            |\\.[0-9_]+(?:[eE][-+][0-9]+)?
            |[-+]?[0-9][0-9_]*(?::[0-5]?[0-9])+\\.[0-9_]*
            |[-+]?\\.(?:inf|Inf|INF)
            |\\.(?:nan|NaN|NAN))$''', re.X),
            list(u'-+0123456789.'))
        data = yaml.load(Path(yaml_file).read_text(), Loader=loader)
        outputs = []
        for dtype in self.dataclass_types:
            keys = {f.name for f in dataclasses.fields(dtype) if f.init}
            arg_name = dtype.__mro__[-2].__name__
            inputs = {k: v for k, v in data[arg_name].items() if k in keys}
            obj = dtype(**inputs)
            outputs.append(obj)
        return (*outputs,)


DataClassType = NewType("DataClassType", Any)
logger = logging.get_logger(__name__)


class SketchReader(BaseReader):
    name: str = "sketch"

    def postprocess(
        self,
        output: Union[np.ndarray, EvalLoopOutput],
        eval_examples: datasets.Dataset,
        eval_dataset: datasets.Dataset,
        mode: str = "evaluate",
    ) -> Union[EvalPrediction, Dict[str, float]]:
        # External Front Verification (E-FV)
        if isinstance(output, EvalLoopOutput):
            logits = output.predictions
        else:
            logits = output
        example_id_to_index = {k: i for i, k in enumerate(eval_examples[ID_COLUMN_NAME])}
        features_per_example = collections.defaultdict(list)
        for i, feature in enumerate(eval_dataset):
            features_per_example[example_id_to_index[feature["example_id"]]].append(i)

        count_map = {k: len(v) for k, v in features_per_example.items()}

        logits_ans = np.zeros(len(count_map))
        logits_na = np.zeros(len(count_map))
        for example_index, example in enumerate(tqdm(eval_examples)):
            feature_indices = features_per_example[example_index]
            n_strides = count_map[example_index]
            logits_ans[example_index] += logits[example_index, 0] / n_strides
            logits_na[example_index] += logits[example_index, 1] / n_strides

        # Calculate E-FV score
        score_ext = logits_ans - logits_na

        # Save external front verification score
        final_map = dict(zip(eval_examples[ID_COLUMN_NAME], score_ext.tolist()))
        with open(os.path.join(self.args.output_dir, SCORE_EXT_FILE_NAME), "w") as writer:
            writer.write(json.dumps(final_map, indent=4) + "\n")
        if mode == "evaluate":
            return EvalPrediction(
                predictions=logits, label_ids=output.label_ids,
            )
        else:
            return final_map


class IntensiveReader(BaseReader):
    name: str = "intensive"

    def postprocess(
        self,
        output: EvalLoopOutput,
        eval_examples: datasets.Dataset,
        eval_dataset: datasets.Dataset,
        log_level: int = logging.WARNING,
        mode: str = "evaluate",
    ) -> Union[List[Dict[str, Any]], EvalPrediction]:
        # Internal Front Verification (I-FV)
        # Verification is already done inside the model
        # Post-processing: we match the start logits and end logits to answers in the original context.
        predictions, nbest_json, scores_diff_json = self.compute_predictions(
            eval_examples,
            eval_dataset,
            output.predictions,
            version_2_with_negative=self.data_args.version_2_with_negative,
            n_best_size=self.data_args.n_best_size,
            max_answer_length=self.data_args.max_answer_length,
            null_score_diff_threshold=self.data_args.null_score_diff_threshold,
            output_dir=self.args.output_dir,
            log_level=log_level,
            n_tops=(self.data_args.start_n_top, self.data_args.end_n_top),
        )
        if mode == "retro_inference":
            return nbest_json, scores_diff_json
        # Format the result to the format the metric expects.
        if self.data_args.version_2_with_negative:
            formatted_predictions = [
                {"id": k, "prediction_text": v, "no_answer_probability": scores_diff_json[k]}
                for k, v in predictions.items()
            ]
        else:
            formatted_predictions = [
                {"id": k, "prediction_text": v}
                for k, v in predictions.items()
            ]
        if mode == "predict":
            return formatted_predictions
        else:
            references = [
                {"id": ex[ID_COLUMN_NAME], "answers": ex[ANSWER_COLUMN_NAME]}
                for ex in eval_examples
            ]
            return EvalPrediction(
                predictions=formatted_predictions, label_ids=references
            )

    def compute_predictions(
        self,
        examples: datasets.Dataset,
        features: datasets.Dataset,
        predictions: Tuple[np.ndarray, np.ndarray],
        version_2_with_negative: bool = False,
        n_best_size: int = 20,
        max_answer_length: int = 30,
        null_score_diff_threshold: float = 0.0,
        output_dir: Optional[str] = None,
        log_level: Optional[int] = logging.WARNING,
        n_tops: Tuple[int, int] = (-1, -1),
        use_choice_logits: bool = False,
    ):
        # Threshold-based Answerable Verification (TAV)
        if len(predictions) not in [2, 3]:
            raise ValueError("`predictions` should be a tuple with two or three elements "
                             "(start_logits, end_logits, choice_logits).")
        all_start_logits, all_end_logits = predictions[:2]
        all_choice_logits = None
        if len(predictions) == 3:
            all_choice_logits = predictions[-1]

        # Build a map example to its corresponding features.
        example_id_to_index = {k: i for i, k in enumerate(examples[ID_COLUMN_NAME])}
        features_per_example = collections.defaultdict(list)
        for i, feature in enumerate(features):
            features_per_example[example_id_to_index[feature["example_id"]]].append(i)

        all_predictions = collections.OrderedDict()
        all_nbest_json = collections.OrderedDict()
        scores_diff_json = collections.OrderedDict() if version_2_with_negative else None

        # Logging.
        logger.setLevel(log_level)
        logger.info(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

        # Let's loop over all the examples!
        for example_index, example in enumerate(tqdm(examples)):
            # Those are the indices of the features associated to the current example.
            feature_indices = features_per_example[example_index]

            min_null_prediction = None
            prelim_predictions = []

            # Looping through all the features associated to the current example.
            for feature_index in feature_indices:
                # We grab the predictions of the model for this feature.
                start_logits = all_start_logits[feature_index]
                end_logits = all_end_logits[feature_index]
                # score_null = s1 + e1
                feature_null_score = start_logits[0] + end_logits[0]
                if all_choice_logits is not None:
                    choice_logits = all_choice_logits[feature_index]
                if use_choice_logits:
                    feature_null_score = choice_logits[1]
                # This is what will allow us to map some the positions
                # in our logits to span of texts in the original context.
                offset_mapping = features[feature_index]["offset_mapping"]
                # Optional `token_is_max_context`,
                # if provided we will remove answers that do not have the maximum context
                # available in the current feature.
                token_is_max_context = features[feature_index].get("token_is_max_context", None)

                # Update minimum null prediction.
                if (
                    min_null_prediction is None or
                    min_null_prediction["score"] > feature_null_score
                ):
                    min_null_prediction = {
                        "offsets": (0, 0),
                        "score": feature_null_score,
                        "start_logit": start_logits[0],
                        "end_logit": end_logits[0],
                    }

                # Go through all possibilities for the {top k} greater start and end logits
                start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
                end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Don't consider out-of-scope answers!
                        # either because the indices are out of bounds
                        # or correspond to part of the input_ids that are note in the context.
                        if (
                            start_index >= len(offset_mapping)
                            or end_index >= len(offset_mapping)
                            or not offset_mapping[start_index]
                            or not offset_mapping[end_index]
                        ):
                            continue
                        # Don't consider answers with a length negative or > max_answer_length.
                        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                            continue
                        # Don't consider answer that don't have the maximum context available
                        # (if such information is provided).
                        if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                            continue
                        prelim_predictions.append(
                            {
                                "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                                "score": start_logits[start_index] + end_logits[end_index],
                                "start_logit": start_logits[start_index],
                                "end_logit": end_logits[end_index],
                            }
                        )

            if version_2_with_negative:
                # Add the minimum null prediction
                prelim_predictions.append(min_null_prediction)
                null_score = min_null_prediction["score"]

            # Only keep the best `n_best_size` predictions
            predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

            # Add back the minimum null prediction if it was removed because of its low score.
            if version_2_with_negative and not any(p["offsets"] == (0, 0) for p in predictions):
                predictions.append(min_null_prediction)

            # Use the offsets to gather the answer text in the original context.
            context = example["context"]

            final_pred.append((predictions[0]['offsets'][0], predictions[0]['offsets'][1]))
            for pred in predictions:
                offsets = pred["offsets"]

                pred["text"] = context[offsets[0] : offsets[1]]
            print(final_pred)
            # In the very rare edge case we have not a single non-null prediction,
            # we create a fake prediction to avoid failure.
            if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
                predictions.insert(0, {"text": "", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0,})

            # Compute the softmax of all scores
            # (we do it with numpy to stay independent from torch/tf) in this file,
            #  using the LogSum trick).
            scores = np.array([pred.pop("score") for pred in predictions])
            exp_scores = np.exp(scores - np.max(scores))
            probs = exp_scores / exp_scores.sum()

            # Include the probabilities in our predictions.
            for prob, pred in zip(probs, predictions):
                pred["probability"] = prob

            # Pick the best prediction. If the null answer is not possible, this is easy.
            if not version_2_with_negative:
                all_predictions[example[ID_COLUMN_NAME]] = predictions[0]["text"]
            else:
                # Otherwise we first need to find the best non-empty prediction.
                i = 0
                try:
                    while predictions[i]["text"] == "":
                        i += 1
                except:
                    i = 0
                best_non_null_pred = predictions[i]

                # Then we compare to the null prediction using the threshold.
                score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
                scores_diff_json[example[ID_COLUMN_NAME]] = float(score_diff)  # To be JSON-serializable.
                if score_diff > null_score_diff_threshold:
                    all_predictions[example[ID_COLUMN_NAME]] = ""
                else:
                    all_predictions[example[ID_COLUMN_NAME]] = best_non_null_pred["text"]

            # Make `predictions` JSON-serializable by casting np.float back to float.
            all_nbest_json[example[ID_COLUMN_NAME]] = [
                {k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items()}
                for pred in predictions
            ]

        # If we have an output_dir, let's save all those dicts.
        if output_dir is not None:
            if not os.path.isdir(output_dir):
                raise EnvironmentError(f"{output_dir} is not a directory.")

            prediction_file = os.path.join(output_dir, INTENSIVE_PRED_FILE_NAME)
            nbest_file = os.path.join(output_dir, NBEST_PRED_FILE_NAME)
            if version_2_with_negative:
                null_odds_file = os.path.join(output_dir, SCORE_DIFF_FILE_NAME)

            logger.info(f"Saving predictions to {prediction_file}.")
            with open(prediction_file, "w") as writer:
                writer.write(json.dumps(all_predictions, indent=4) + "\n")
            logger.info(f"Saving nbest_preds to {nbest_file}.")
            with open(nbest_file, "w") as writer:
                writer.write(json.dumps(all_nbest_json, indent=4) + "\n")
            if version_2_with_negative:
                logger.info(f"Saving null_odds to {null_odds_file}.")
                with open(null_odds_file, "w") as writer:
                    writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

        return all_predictions, all_nbest_json, scores_diff_json


class RearVerifier:

    def __init__(
        self,
        beta1: int = 1,
        beta2: int = 1,
        best_cof: int = 1,
        thresh: float = 0.0,
    ):
        self.beta1 = beta1
        self.beta2 = beta2
        self.best_cof = best_cof
        self.thresh = thresh

    def __call__(
        self,
        score_ext: Dict[str, float],
        score_diff: Dict[str, float],
        nbest_preds: Dict[str, Dict[int, Dict[str, float]]]
    ):
        all_scores = collections.OrderedDict()
        assert score_ext.keys() == score_diff.keys()
        for key in score_ext.keys():
            if key not in all_scores:
                all_scores[key] = []
            all_scores[key].extend(
                [self.beta1 * score_ext[key],
                 self.beta2 * score_diff[key]]
            )
        output_scores = {}
        for key, scores in all_scores.items():
            mean_score = sum(scores) / float(len(scores))
            output_scores[key] = mean_score

        all_nbest = collections.OrderedDict()
        for key, entries in nbest_preds.items():
            if key not in all_nbest:
                all_nbest[key] = collections.defaultdict(float)
            for entry in entries:
                prob = self.best_cof * entry["probability"]
                all_nbest[key][entry["text"]] += prob

        output_predictions = {}
        for key, entry_map in all_nbest.items():
            sorted_texts = sorted(
                entry_map.keys(), key=lambda x: entry_map[x], reverse=True
            )
            best_text = sorted_texts[0]
            output_predictions[key] = best_text

        for qid in output_predictions.keys():
            if output_scores[qid] > self.thresh:
                output_predictions[qid] = ""

        return output_predictions, output_scores



class RetroReader:

    def __init__(
        self,
        args,
        sketch_reader: SketchReader,
        intensive_reader: IntensiveReader,
        rear_verifier: RearVerifier,
        prep_fn: Tuple[Callable, Callable],
    ):
        self.args = args
        # Set submodules
        self.sketch_reader = sketch_reader
        self.intensive_reader = intensive_reader
        self.rear_verifier = rear_verifier

        # Set prep function for inference
        self.sketch_prep_fn, self.intensive_prep_fn = prep_fn

    @classmethod
    def load(
        cls,
        train_examples=None,
        sketch_train_dataset=None,
        intensive_train_dataset=None,
        eval_examples=None,
        sketch_eval_dataset=None,
        intensive_eval_dataset=None,
        config_file: str = '/content/default_config.yaml',
    ):
        # Get arguments from yaml files
        parser = HfArgumentParser([RetroArguments, TrainingArguments])
        retro_args, training_args = parser.parse_yaml_file(yaml_file=config_file)
        if training_args.run_name is not None and "," in training_args.run_name:
            sketch_run_name, intensive_run_name = training_args.run_name.split(",")
        else:
            sketch_run_name, intensive_run_name = None, None
        if training_args.metric_for_best_model is not None and "," in training_args.metric_for_best_model:
            sketch_best_metric, intensive_best_metric = training_args.metric_for_best_model.split(",")
        else:
            sketch_best_metric, intensive_best_metric = None, None
        sketch_training_args = copy.deepcopy(training_args)
        intensive_training_args = training_args

        # If `train_examples` is feeded, perform preprocessing
        if train_examples is not None and sketch_train_dataset is None:
            sketch_prep_fn, is_batched = get_sketch_features(sketch_tokenizer, "train", retro_args)
            sketch_train_dataset = train_examples.map(
                sketch_prep_fn,
                batched=is_batched,
                remove_columns=train_examples.column_names,
                num_proc=retro_args.preprocessing_num_workers,
                load_from_cache_file=not retro_args.overwrite_cache,
            )
        # If `eval_examples` is feeded, perform preprocessing
        if eval_examples is not None and sketch_eval_dataset is None:
            sketch_prep_fn, is_batched = get_sketch_features(sketch_tokenizer, "eval", retro_args)
            sketch_eval_dataset = eval_examples.map(
                sketch_prep_fn,
                batched=is_batched,
                remove_columns=eval_examples.column_names,
                num_proc=retro_args.preprocessing_num_workers,
                load_from_cache_file=not retro_args.overwrite_cache,
            )
        # Get preprocessing function for inference
        sketch_prep_fn, _ = get_sketch_features(sketch_tokenizer, "test", retro_args)

        # Get model for sketch reader


        # Get sketch reader
        sketch_training_args.run_name = sketch_run_name
        sketch_training_args.output_dir += "/sketch"
        sketch_training_args.metric_for_best_model = sketch_best_metric
        sketch_reader = SketchReader(
            model=sketch_model,
            args=sketch_training_args,
            train_dataset=sketch_train_dataset,
            eval_dataset=sketch_eval_dataset,
            eval_examples=eval_examples,
            data_args=retro_args,
            tokenizer=sketch_tokenizer,
            compute_metrics=compute_classification_metric,
        )

        # If `train_examples` is feeded, perform preprocessing
        if train_examples is not None and intensive_train_dataset is None:
            intensive_prep_fn, is_batched = get_intensive_features(intensive_tokenizer, "train", retro_args)
            intensive_train_dataset = train_examples.map(
                intensive_prep_fn,
                batched=is_batched,
                remove_columns=train_examples.column_names,
                num_proc=retro_args.preprocessing_num_workers,
                load_from_cache_file=not retro_args.overwrite_cache,
            )
        # If `eval_examples` is feeded, perform preprocessing
        if eval_examples is not None and intensive_eval_dataset is None:
            intensive_prep_fn, is_batched = get_intensive_features(intensive_tokenizer, "eval", retro_args)
            intensive_eval_dataset = eval_examples.map(
                intensive_prep_fn,
                batched=is_batched,
                remove_columns=eval_examples.column_names,
                num_proc=retro_args.preprocessing_num_workers,
                load_from_cache_file=not retro_args.overwrite_cache,
            )
        # Get preprocessing function for inferencem
        intensive_prep_fn, _ = get_intensive_features(intensive_tokenizer, "test", retro_args)

        # Get model for intensive reader

        # Get intensive reader
        intensive_training_args.run_name = intensive_run_name
        intensive_training_args.output_dir += "/intensive"
        intensive_training_args.metric_for_best_model = intensive_best_metric
        intensive_reader = IntensiveReader(
            model=intensive_model,
            args=intensive_training_args,
            train_dataset=intensive_train_dataset,
            eval_dataset=intensive_eval_dataset,
            eval_examples=eval_examples,
            data_args=retro_args,
            tokenizer=intensive_tokenizer,
            compute_metrics=compute_squad_v2,
        )

        # Get rear verifier
        rear_verifier = RearVerifier(
            beta1=retro_args.beta1,
            beta2=retro_args.beta2,
            best_cof=retro_args.best_cof,
            thresh=retro_args.rear_threshold,
        )

        return cls(
            args=retro_args,
            sketch_reader=sketch_reader,
            intensive_reader=intensive_reader,
            rear_verifier=rear_verifier,
            prep_fn=(sketch_prep_fn, intensive_prep_fn),
        )

    def __call__(
        self,
        query: str,
        context: Union[str, List[str]],
        return_submodule_outputs: bool = False,
    ) -> Tuple[Any]:
        if isinstance(context, list):
            context = " ".join(context)
        predict_examples = datasets.Dataset.from_dict({
            "example_id": ["0"],
            ID_COLUMN_NAME: ["id-01"],
            QUESTION_COLUMN_NAME: [query],
            CONTEXT_COLUMN_NAME: [context]
        })
        return self.inference(predict_examples, return_submodule_outputs)

    def train(self, module: str = "all"):

        def wandb_finish(module):
            for callback in module.callback_handler.callbacks:
                if "wandb" in str(type(callback)).lower():
                    callback._wandb.finish()
                    callback._initialized = False

        # Train sketch reader
        if module.lower() in ["all", "sketch"]:
            self.sketch_reader.train()
            self.sketch_reader.save_model()
            self.sketch_reader.save_state()
            self.sketch_reader.free_memory()
            wandb_finish(self.sketch_reader)
        # Train intensive reader
        if module.lower() in ["all", "intensive"]:
            self.intensive_reader.train()
            self.intensive_reader.save_model()
            self.intensive_reader.save_state()
            self.intensive_reader.free_memory()
            wandb_finish(self.intensive_reader)

    def inference(self, test_dataset: datasets.Dataset, return_submodule_outputs: bool) -> Tuple[Any]:
        if "example_id" not in test_dataset.column_names:
            test_dataset = test_dataset.map(
                lambda _, i: {"example_id": str(i)},
                with_indices=True,
            )
        sketch_features = test_dataset.map(
            self.sketch_prep_fn,
            batched=True,
            remove_columns=test_dataset.column_names,
        )
        intensive_features = test_dataset.map(
            self.intensive_prep_fn,
            batched=True,
            remove_columns=test_dataset.column_names,
        )
        #self.sketch_reader.to(self.sketch_reader.args.device)
        score_ext = self.sketch_reader.predict(sketch_features, test_dataset)
        #self.sketch_reader.to("cpu")
        #self.intensive_reader.to(self.intensive_reader.args.device)
        nbest_preds, score_diff = self.intensive_reader.predict(
            intensive_features, test_dataset, mode="retro_inference")
        #self.intensive_reader.to("cpu")
        predictions, scores = self.rear_verifier(score_ext, score_diff, nbest_preds)
        outputs = (predictions, scores)
        if return_submodule_outputs:
            outputs += (score_ext, nbest_preds, score_diff)
        return outputs

    @property
    def null_score_diff_threshold(self):
        return self.args.null_score_diff_threshold

    @null_score_diff_threshold.setter
    def null_score_diff_threshold(self, val):
        self.args.null_score_diff_threshold = val

    @property
    def n_best_size(self):
        return self.args.n_best_size

    @n_best_size.setter
    def n_best_size(self, val):
        self.args.n_best_size = val

    @property
    def beta1(self):
        return self.rear_verifier.beta1

    @beta1.setter
    def beta1(self, val):
        self.rear_verifier.beta1 = val

    @property
    def beta2(self):
        return self.rear_verifier.beta2

    @beta2.setter
    def beta2(self, val):
        self.rear_verifier.beta2 = val

    @property
    def best_cof(self):
        return self.rear_verifier.best_cof

    @best_cof.setter
    def best_cof(self, val):
        self.rear_verifier.best_cof = val

    @property
    def rear_threshold(self):
        return self.rear_verifier.thresh

    @rear_threshold.setter
    def rear_threshold(self, val):
        self.rear_verifier.thresh = val


inference_en_electra_large="""
RetroDataModelArguments:

    # DataArguments
    max_seq_length: 256
    max_answer_length: 18
    doc_stride: 64
    return_token_type_ids: True
    pad_to_max_length: True
    preprocessing_num_workers: 128
    overwrite_cache: True
    version_2_with_negative: True
    null_score_diff_threshold: 0.0
    rear_threshold: 0.0
    n_best_size: 20
    use_choice_logits: False
    start_n_top: -1
    end_n_top: -1
    beta1: 1
    beta2: 1
    best_cof: 1

    # ModelArguments
    use_auth_token: False

    # SketchModelArguments
    sketch_revision: en-electra-large-sketch
    sketch_model_name: /content/drive/MyDrive/FinalCompilation/retro-reader
    sketch_architectures: ElectraForSequenceClassification

    # IntensiveModelArguments
    intensive_revision: en-electra-large-intensive
    intensive_model_name: /content/drive/MyDrive/FinalCompilation/retro-reader
    intensive_architectures: ElectraForQuestionAnsweringAVPool


TrainingArguments:
    output_dir: outputs
    no_cuda: True # If you want to use cuda,
                  # change `no_cuda` to False and `fp16` to True
    per_device_train_batch_size: 1
    per_device_eval_batch_size: 12

"""
yaml_electra=yaml.safe_load(inference_en_electra_large)
with open("inference_en_electra_large.yaml", "w") as fh:
  yaml.dump(yaml_electra, fh)

model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-TinyBERT-L-2-v2')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-TinyBERT-L-2-v2')

class PassageSimilarity:
  def __init__(self, nlp, data):
    self.tokenize = lambda text: [token.lemma_ for token in nlp(text)]
    self.bm25 = None
    self.passages = None
    self.data = data
    self.avg_idf = None
    self.corpus = None

  def fit(self):
    print("inside")
    corpus = [self.tokenize(self.data["paragraph"][p]) for p in range(len(self.data))]
    self.corpus = corpus
    bm25 = BM25(corpus)
    average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())
    self.bm25 = bm25
    self.avg_idf = average_idf

  def most_similar(self, question):
    tokens = self.tokenize(question)
    scores = self.bm25.get_scores(tokens, average_idf = self.avg_idf)
    pairs = [(s, i) for i, s in enumerate(scores)]
    pairs.sort(reverse = True)
    x=[]
    for i in range(len(pairs)):
      x.append(pairs[i][1])
    s = [pairs[0][1],pairs[1][1],pairs[2][1],pairs[3][1],pairs[4][1],pairs[5][1],pairs[6][1],pairs[7][1],pairs[8][1],pairs[9][1]] if len(pairs)>10 else x
    return s

SPACY_MODEL = os.environ.get('SPACY_MODEL', 'en_core_web_sm')
nlp = spacy.load(SPACY_MODEL, disable=["ner","parser","textcat"])
passage_retriever = PassageSimilarity(nlp, data_para)
passage_retriever.fit()

config_file = "inference_en_electra_large.yaml"
parser = HfArgumentParser([RetroArguments, TrainingArguments])
retro_args, training_args = parser.parse_yaml_file(yaml_file=config_file)
sketch_tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path=retro_args.sketch_tokenizer_name,
            use_auth_token=retro_args.use_auth_token,
            revision=retro_args.sketch_revision,
        )
sketch_model_cls = retro_args.sketch_model_cls
sketch_model = sketch_model_cls.from_pretrained(
            pretrained_model_name_or_path=retro_args.sketch_model_name,
            use_auth_token=retro_args.use_auth_token,
            revision=retro_args.sketch_revision,
        )
intensive_tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path=retro_args.intensive_tokenizer_name,
            use_auth_token=retro_args.use_auth_token,
            revision=retro_args.intensive_revision,
        )
intensive_model_cls = retro_args.intensive_model_cls
intensive_model = intensive_model_cls.from_pretrained(
            pretrained_model_name_or_path=retro_args.intensive_model_name,
            use_auth_token=retro_args.use_auth_token,
            revision=retro_args.intensive_revision,
        )

In [ ]:
%%capture
syn_Data = syn_data_former(data_para)

In [ ]:
len(syn_Data)

1179

In [ ]:
syn_df= pd.DataFrame.from_dict(syn_Data, orient='index')
print(syn_df)
syn_df.to_csv("/content/syn_Data.csv",header=False)

                                                     0   \
1     {'What is the name of Apple's iPod?': {'score'...   
2     {'What is the name of a device that has 128GB ...   
3     {'What is the name of Apple's software used to...   
4     {'Music and Videos are two separate types of w...   
5     {'Along with Space grey, gold and silver what ...   
...                                                 ...   
1175  {'What does a customer need to be described or...   
1176  {'What type of records can be created through ...   
1177  {'What is the validity of a PAT when it's crea...   
1178  {'What will you do to create your PAT?': {'sco...   
1179  {'What is the name of your app?': {'score': 0....   

                                                     1   \
1     {'What is the name of a line that makes portab...   
2     {'What is an example of a digital music player...   
3                                                  None   
4     {'What was the first device to use a media pla...

In [ ]:


# # define dictionary
# dict2 = syn_Data

# # create a binary pickle file
# f = open("/content/syn_Data2.pkl","wb")

# # write the python object (dict) to pickle file
# pickle.dump(dict2,f)

# # close file
# f.close()

In [ ]:
file_to_read = open("/content/drive/MyDrive/syn_Data.pkl", "rb")
syn_Data = pickle.load(file_to_read)

In [ ]:
sample_qa_answers=pd.read_csv("/content/drive/MyDrive/question_answers.csv")
for i in range(len(sample_qa_answers)):
  syn_Data[sample_qa_answers["paragraph_id"][i]].append({sample_qa_answers["question"][i]:{"score":1, "start":0, "end":0, "answer":sample_qa_answers["answer"][i]}})

In [ ]:
len(syn_Data)

1179

In [ ]:
syn_Data

{1: [{"What is the name of Apple's iPod?": {'score': 0.5342235565185547,
    'start': 0,
    'end': 8,
    'answer': 'The iPod'}},
  {'What is the name of a line that makes portable media players?': {'score': 0.9492592215538025,
    'start': 0,
    'end': 8,
    'answer': 'The iPod'}},
  {'In what year were the most recent iPod redesigns announced?': {'score': 0.9785794615745544,
    'start': 286,
    'end': 290,
    'answer': '2015'}},
  {'What was the Macintosh version of?': {'score': 0.9977478384971619,
    'start': 187,
    'end': 193,
    'answer': 'iTunes'}},
  {'Which company produces the iPod?': {'score': 1,
    'start': 0,
    'end': 0,
    'answer': 'Apple'}},
  {'When was the original iPod released?': {'score': 1,
    'start': 0,
    'end': 0,
    'answer': 'October 23, 2001'}},
  {'How many different types of iPod are currently available?': {'score': 1,
    'start': 0,
    'end': 0,
    'answer': 'three'}},
  {'What kind of device is the iPod?': {'score': 1,
    'start': 0,

In [ ]:
%%capture
# RetroReader = from_library()
def get_theme_model(theme):
  lang_code="en_XX"
  retro_reader = RetroReader.load(config_file=config_file)
  retro_reader.null_score_diff_threshold=0
  retro_reader.rear_threshold=0
  retro_reader.n_best_size=1
  retro_reader.beta1=1
  retro_reader.beta2=1
  retro_reader.best_cof=1
  return_submodule_outputs = False

  global_model = retro_reader

  # Here, you can load and return a model which is fine tuned for a theme.
  # In case, you only have a global model, you can return that.
  return global_model

In [ ]:
for i in range(len(data_para)):
  data_para['theme'][i] = data_para['theme'][i].lower()

In [ ]:
data_para

,id,paragraph,theme
0,1,The iPod is a line of portable media players a...,ipod
1,2,"Like other digital music players, iPods can se...",ipod
2,3,Apple's iTunes software (and other alternative...,ipod
3,4,"Before the release of iOS 5, the iPod branding...",ipod
4,5,"In mid-2015, a new model of the iPod Touch was...",ipod
...,...,...,...
1174,1175,Adding new Customers# Customer records offer a...,devrev
1175,1176,Apart from customer records that get automatic...,devrev
1176,1177,Authentication to DevRev APIs requires a perso...,devrev
1177,1178,Generate a Personal Access Token (PAT)# In the...,devrev


In [ ]:
%%capture
from prompt_toolkit.widgets.toolbars import fragment_list_len

# Allowed to make changes.

def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]["theme"].lower()
  print(theme)
  local_para_data = data_para[data_para['theme'] == theme].reset_index(drop = True)
  print(local_para_data)
  passage_retriever = PassageSimilarity(nlp, local_para_data)
  passage_retriever.fit()
  max_paragraph = []
  answ = []
  flag = False
  for question in questions:
    lis_para = list()
    ques = question["question"]
    for para_data in range(len(local_para_data)):
      score, ans2 = comparisons(ques, local_para_data['id'][para_data], syn_Data)
      if (score == None):
        continue
      else:
        lis_para.append((score, ans2, local_para_data['id'][para_data]))
    if (len(lis_para) == 0):
      flag = True
    else:
      lis_para.sort(reverse = True)
      max_paragraph.append(lis_para[0][2])
      answ.append(lis_para[0][1])

    if (flag == True):
      para_id = passage_retriever.most_similar(ques)
      ka=[]
      for j in para_id:
        features = tokenizer([ques], [local_para_data['paragraph'][j]],  padding=True, truncation=True, return_tensors="pt")
        model.eval()
        with torch.no_grad():
            scores = model(**features).logits
            ka.append((scores,j))
      if len(ka)>1:
        max1 = -100
        max2 = -100
        ka.sort(reverse = True)
        ans1 = ka[0][1]
        ans2 = ka[1][1]
        top3 = [local_para_data["paragraph"][para_id[0]]] if para_id[0] == ans1 or para_id[0] == ans2 else [local_para_data["paragraph"][ans1],local_para_data["paragraph"][ans2],local_para_data["paragraph"][para_id[0]]]
      elif len(ka)==1:
        top3 = [data_para["paragraph"][para_id[0]]]
      lists = [ques,top3]

      query=lists[0]
      context = ""
      if len(lists[1])==1:
        pass
      else:
        a = len(lists[1][0])
        b = a + len(lists[1][1])
      for members in lists[1]:
        context+=members + " "

      outputs = theme_model(query=query,context=context,return_submodule_outputs=True)
      answer, score = outputs[0]["id-01"], outputs[1]
      score_ext, nbest_preds, score_diff = outputs[2:]
      print('///////////////////////////////final_pred', final_pred)
      start_index=final_pred[0][0]
      final_index=final_pred[0][1]

      if(start_index==final_index):
        max_paragraph.append(-1)
      else:
        if len(lists[1])!=1:
          if(start_index>=b):
            pid = int(local_para_data[local_para_data['paragraph'] == lists[1][2]]['id'])
            max_paragraph.append(pid)
          elif(start_index<=a):
            pid = int(local_para_data[local_para_data['paragraph'] == lists[1][0]]['id'])
            max_paragraph.append(pid)
          else:
            pid = int(local_para_data[local_para_data['paragraph'] == lists[1][1]]['id'])
            max_paragraph.append(pid)
        else:
          pid = int(local_para_data[local_para_data['paragraph'] == lists[1][0]]['id'])
          max_paragraph.append(pid)

      final_pred.pop()

      answ.append(answer)

    ans={}
    ans["question_id"] = question['id']
    for i in range(len(answ)):
      ans["answers"]="["+answ[i]+"]"
      ans["paragraph_id"]=max_paragraph[i]

    pred_out.append(ans)

from prompt_toolkit.widgets.toolbars import fragment_list_len

# Allowed to make changes.

def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]["theme"].lower()
  print(theme)
  local_para_data = data_para[data_para['theme'] == theme].reset_index(drop = True)
  print(local_para_data)
  passage_retriever = PassageSimilarity(nlp, local_para_data)
  passage_retriever.fit()
  max_paragraph = []
  answ = []
  flag = False
  for question in questions:
    lis_para = list()
    ques = question["question"]
    for para_data in range(len(local_para_data)):
      score, ans2 = comparisons(ques, local_para_data['id'][para_data], syn_Data)
      if (score == None):
        continue
      else:
        lis_para.append((score, ans2, local_para_data['id'][para_data]))
    if (len(lis_para) == 0):
      flag = True
    else:
      lis_para.sort(reverse = True)
      max_paragraph.append(lis_para[0][2])
      answ.append(lis_para[0][1])

    if (flag == True):
      para_id = passage_retriever.most_similar(ques)
      ka=[]
      for j in para_id:
        features = tokenizer([ques], [local_para_data['paragraph'][j]],  padding=True, truncation=True, return_tensors="pt")
        model.eval()
        with torch.no_grad():
            scores = model(**features).logits
            ka.append((scores,j))
      if len(ka)>1:
        max1 = -100
        max2 = -100
        ka.sort(reverse = True)
        ans1 = ka[0][1]
        ans2 = ka[1][1]
        top3 = [local_para_data["paragraph"][para_id[0]]] if para_id[0] == ans1 or para_id[0] == ans2 else [local_para_data["paragraph"][ans1],local_para_data["paragraph"][ans2],local_para_data["paragraph"][para_id[0]]]
      elif len(ka)==1:
        top3 = [data_para["paragraph"][para_id[0]]]
      lists = [ques,top3]

      query=lists[0]
      context = ""
      if len(lists[1])==1:
        pass
      else:
        a = len(lists[1][0])
        b = a + len(lists[1][1])
      for members in lists[1]:
        context+=members + " "

      outputs = theme_model(query=query,context=context,return_submodule_outputs=True)
      answer, score = outputs[0]["id-01"], outputs[1]
      score_ext, nbest_preds, score_diff = outputs[2:]
      print('///////////////////////////////final_pred', final_pred)
      start_index=final_pred[0][0]
      final_index=final_pred[0][1]

      if(start_index==final_index):
        max_paragraph.append(-1)
      else:
        if len(lists[1])!=1:
          if(start_index>=b):
            pid = int(local_para_data[local_para_data['paragraph'] == lists[1][2]]['id'])
            max_paragraph.append(pid)
          elif(start_index<=a):
            pid = int(local_para_data[local_para_data['paragraph'] == lists[1][0]]['id'])
            max_paragraph.append(pid)
          else:
            pid = int(local_para_data[local_para_data['paragraph'] == lists[1][1]]['id'])
            max_paragraph.append(pid)
        else:
          pid = int(local_para_data[local_para_data['paragraph'] == lists[1][0]]['id'])
          max_paragraph.append(pid)

      final_pred.pop()

      answ.append(answer)

    ans={}
    ans["question_id"] = question['id']
    for i in range(len(answ)):
      if answ[i] == '':
        ans['answers'] = answ[i]
      else:
        ans["answers"]="["+answ[i]+"]"
      ans["paragraph_id"]=max_paragraph[i]

    pred_out.append(ans)


In [ ]:
%%capture
# NOT allowed to make changes.

# All theme prediction.
questions = json.loads(pd.read_csv("/content/drive/MyDrive/sample_input_question_1.csv").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv("/content/drive/MyDrive/sample_theme_interval.csv").to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in theme_intervals:
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
pred_df.fillna(value='', inplace=True)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('sample_output_prediction.csv', index=False)

KeyboardInterrupt: ignored

In [ ]:
# NOT allowed to make changes.

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  if len(ground_truths) == 0:
    return len(predicted) == 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

In [ ]:
# NOT allowed to make changes.

# Evaluation methodology.
metrics = {}
pred = pd.read_csv("sample_output_prediction.csv")
pred.fillna(value='', inplace=True)
truth = pd.read_csv("/content/drive/MyDrive/Resource/sample_truth.csv")
truth.fillna(value='', inplace=True)
truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv("/content/drive/MyDrive/Resource/sample_question.csv")
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]

  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  if predicted_paragraph in truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1

In [ ]:
# NOT allowed to make changes.

# Final score.
inf_time_threshold = 1000.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0

fps=0.0
fqs=0.0

# Weight would stay hidden from teams.
theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}
for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"]
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_para_score += theme_weights[theme] * inf_time_score * para_score
  fqs+= theme_weights[theme] * qa_score
  fps+= theme_weights[theme] * para_score
print (final_para_score)
print (final_qa_score)
print(fqs)
print(fps)


In [ ]:
pred_out

In [ ]:
pred_out

[{'question_id': 1, 'answers': '[Apple]', 'paragraph_id': 34},
 {'question_id': 2, 'answers': '[October 23, 2001]', 'paragraph_id': 1},
 {'question_id': 3, 'answers': '[5]', 'paragraph_id': 5},
 {'question_id': 4, 'answers': '[business]', 'paragraph_id': 47},
 {'question_id': 5, 'answers': '[user]', 'paragraph_id': 25},
 {'question_id': 6, 'answers': '', 'paragraph_id': -1},
 {'question_id': 7, 'answers': '[controlling the volume]', 'paragraph_id': 25},
 {'question_id': 8, 'answers': '[128 GB]', 'paragraph_id': 2},
 {'question_id': 9, 'answers': '[128 GB]', 'paragraph_id': 2},
 {'question_id': 10, 'answers': '[iPod Shuffle]', 'paragraph_id': 2},
 {'question_id': 11, 'answers': '[iPod Shuffle]', 'paragraph_id': 2},
 {'question_id': 12, 'answers': '[2 GB]', 'paragraph_id': 2},
 {'question_id': 13, 'answers': '[128 GB]', 'paragraph_id': 2},
 {'question_id': 14, 'answers': '[iTunes]', 'paragraph_id': 3},
 {'question_id': 15,
  'answers': '[Apple Macintosh and Microsoft Windows]',
  'paragr

In [ ]:
pred_df.to_csv('/content/sample_output_prediction0.6-with-sample.csv', index=False)

In [ ]:
len(pred_df)

3917

In [ ]:
len(pred_df[pred_df["paragraph_id"]==-1])

1417

In [ ]:
pred_df[pred_df['paragraph_id'] == -1]

,question_id,answers,paragraph_id
5,6,,-1
26,27,,-1
94,95,,-1
102,103,,-1
104,105,,-1
...,...,...,...
3905,5236,,-1
3908,5239,,-1
3909,5240,,-1
3910,5241,,-1


In [ ]:
total = 0
for i in theme_inf_time.keys() :
  total+= theme_inf_time[i]
total/3918

730.0663634170478

3917